#### Setup: Import the InatUtils class, pointed at the proper in/out folders.
Default parameters:
<br>
| Param           | Type          | Default Value  |
|-----------------|---------------|----------------|
| photo_dir       | str           | "in_photos"    |
| gpx_dir         | str           | "in_gpx"       |
| output_dir      | str           | "out_photos"   |
| gmt_offset      | int           | -8             |
| token           | str           | None           |
| trusted_genera  | list          | []             |
| log_level       | str           | "INFO"         |
| min_score       | int \| float  | 75             |

#### 1. Georeference photos
If you've pointed the tool at your GPX and photos, it will try to georeference them on startup (see below)

In [1]:
from inatutils import InatUtils

iu = InatUtils()
print(f"{iu.georeferenced_percent}% of observations are georeferenced:")
for op in iu.out_photos:
    print('\t', op.name)

INFO:id:refresh_token:20:getting token...


modification complete
image written to f:\SGP\repos\inat-upload-utils\out_photos\IMG_8804_georeferenced.JPEG


33.33333333333333% of observations are georeferenced
	 IMG_8804_georeferenced.JPEG


#### 2. Identify Images
Once the images have been loaded, calling `iu.identify()` will iteratively submit them to the iNaturalist CV module. It will reject all results scored lower than its configured `min_score` property, which you can configure.

Sometimes, the tool will be able to use a common ancestor of the results, if no result meets the `min_score` requirement, e.g. fall back on Genus if it can't decide on Species, similar to the iNat upload UI behavior.

Georeferenced photos in `out_photos/` inherit any successful IDs of their parent/source images. The `in_photos` property, derived from `in_photos/` is used as the source of truth when tracking georeferencing/ID/upload status per image, while the `out_photos` property contains data to images that are ready to upload.


In [2]:
iu.min_score = 50
iu.identify()

print(f"\n\n{iu.identified_percent}% of observations are identified:")

for p in iu.in_photos:
    print(f'\timage:{p.name}\n\t\t{p.identity}')

print(f"child images the above have inherited IDs")
for p in iu.out_photos:
    print(f'\timage:{p.name}\n\t\t{p.identity}')


INFO:id:interpret_results:75:acceptable ID found within confidence threshold 50:
	score: 77.88450121879578
	rank: species
	name: Mopalia muscosa
	wiki: http://en.wikipedia.org/wiki/Mopalia_muscosa
INFO:id:interpret_results:88:no acceptable ID found within confidence threshold 50:
	best score: 18.84550005197525
	rank: species
	name: Picea rubens
	wiki: http://en.wikipedia.org/wiki/Picea_rubens
INFO:id:interpret_results:95:resorting to common ancestor of all 10 results:
	rank: genus
	name: Picea
INFO:id:interpret_results:88:no acceptable ID found within confidence threshold 50:
	best score: 37.61661648750305
	rank: species
	name: Calicium glaucellum
	wiki: http://en.wikipedia.org/wiki/Calicium_glaucellum


66.66666666666667% of observations are identified
gfhd32343j43334.jpg 
	 {'name': 'Mopalia muscosa', 'rank': 'species', 'score': 77.88450121879578, 'wiki': 'http://en.wikipedia.org/wiki/Mopalia_muscosa'}
IMG_8782.jpeg 
	 {'name': 'Picea', 'rank': 'genus', 'score': None, 'wiki': 'http://en.wikipedia.org/wiki/Spruce'}
IMG_8804.JPG 
	 {}
IMG_8804_georeferenced.JPEG 
	 {}


#### 3. Upload Images

...work in progress 🐛🐛🐛...

I'm thinking of two approaches: 
1. name IDd as their IDs + "_georeferenced" if geotag succeeded, send that to the endpoint for uploading single images, and provide a url to the user for the final confirmation/data enrichment/button click the automated info before hitting submit.

2. make a csv like the one outlined in [issue #1](https://github.com/SamGartrell/inat-upload-utils/issues/1), with image IDs as tags, then have another tool the user can run to stitch photos to existing observations using the photo ID in the observation tags. This could rely on the existing inaturalist python library.

Challenges:
1. how to know which photos are associated, to prevent multiple uploads of a single organism? See/weigh in on [issue #3](https://github.com/SamGartrell/inat-upload-utils/issues/3)
2. finding the balance between ease-of-upload and respect for the [iNaturalist policy on automation](https://www.inaturalist.org/pages/machine_generated_content)... do we need an upload utility at all?
